In [1]:
import os
import numpy as np
import pandas as pd
import time
from utm import *
from tqdm import tqdm, tqdm_pandas
from osgeo import osr
import coordTransform


In [6]:
DATE = '01' #填写01~30
DAY_BEGIN = '07:00:00'
DAY_END = '22:00:00'
day_begin_name = 700
day_end_name = 2200

ORDER_FILE_PATH_WIN = f'F:/大学/第40期PRP/交通订单数据/traffic_data/gps_201611{DATE}.csv'
DST_DIR_PATH = 'F:/大学/第40期PRP/特征提取/坐标转换后的数据/'

In [7]:


time1 = f'201611{DATE} {DAY_BEGIN}'
time2 = f'201611{DATE} {DAY_END}'
stamp1 = time.mktime(time.strptime(time1, '%Y%m%d %H:%M:%S'))
stamp2 = time.mktime(time.strptime(time2, '%Y%m%d %H:%M:%S'))

print(f'正在导入数据：gps_{DATE}.csv')
#导入原地理数据
df = pd.read_csv(ORDER_FILE_PATH_WIN, header = None) #注意我此处使用的是移动硬盘的地址
df.columns = ['driver_ID', 'order_ID', 'timestamp', 'lon', 'lat']
df.timestamp = df.timestamp + 8*3600
print(f'已导入数据：gps_{DATE}.csv')
## 只取预设时间区间内的数据
df = df[(df['timestamp'] >= stamp1)&(df['timestamp'] < stamp2)].reset_index(drop = True)

# 将空间坐标转换为WGS-84(耗时会很长)
xy = df[['lon','lat']].apply(lambda x: coordTransform.gcj02_to_wgs84(x[0],x[1])[:2], axis = 1)
df['lon'] = [x[0] for x in xy]
df['lat'] = [x[1] for x in xy]
print (f'{DATE} 已生成WGS-84')

# 再把WGS-84转换为UTM平面直角系(保留WGS-84数据)
wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(4326)
# 2.Pseudo-Mercator
inp = osr.SpatialReference()
inp.ImportFromEPSG(3857)
# 3.定义坐标变换映射
transformation = osr.CoordinateTransformation(wgs84, inp)
# 4.转换原数据的坐标
print(f'{DATE} 正在转换原数据坐标')
xy = df[['lon','lat']].apply(lambda x: transformation.TransformPoint(x[0],x[1])[:2], axis = 1)
print(f'{DATE} 已转换原数据坐标')
# 5.写入df
df['x'] = [x[0] for x in xy]
df['y'] = [x[1] for x in xy]
print (f'{DATE} 已生成UTM, 当前数据条数：',len(df))


正在导入数据：gps_01.csv
已导入数据：gps_01.csv
01 已生成WGS-84
01 正在转换原数据坐标
01 已转换原数据坐标
01 已生成UTM, 当前数据条数： 1056734


In [8]:

df.to_csv(DST_DIR_PATH + f'201611{DATE}-{day_begin_name}-{day_end_name}.csv', index = False)
print(f'已写入数据:{DST_DIR_PATH}')

已写入数据:F:/大学/第40期PRP/特征提取/坐标转换后的数据/


In [9]:
df1 = pd.read_csv(DST_DIR_PATH + f'201611{DATE}-{day_begin_name}-{day_end_name}.csv')
df1

,driver_ID,order_ID,timestamp,lon,lat,x,y
0,skvmpkgaj6uxql4owwkepAmajbj8spvc,koCxoglo4-Duog5hprrozrllllbaHoym,1477967048,104.040097,30.680527,1.158169e+07,3.591329e+06
1,skvmpkgaj6uxql4owwkepAmajbj8spvc,koCxoglo4-Duog5hprrozrllllbaHoym,1477967051,104.040227,30.680726,1.158171e+07,3.591354e+06
2,skvmpkgaj6uxql4owwkepAmajbj8spvc,koCxoglo4-Duog5hprrozrllllbaHoym,1477967054,104.040416,30.681046,1.158173e+07,3.591396e+06
3,skvmpkgaj6uxql4owwkepAmajbj8spvc,koCxoglo4-Duog5hprrozrllllbaHoym,1477967057,104.040596,30.681345,1.158175e+07,3.591435e+06
4,skvmpkgaj6uxql4owwkepAmajbj8spvc,koCxoglo4-Duog5hprrozrllllbaHoym,1477967060,104.040765,30.681645,1.158177e+07,3.591473e+06
...,...,...,...,...,...,...,...
1056729,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965343,104.041365,30.660703,1.158183e+07,3.588763e+06
1056730,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965346,104.040956,30.660793,1.158179e+07,3.588775e+06
1056731,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965349,104.040517,30.660874,1.158174e+07,3.588785e+06
1056732,gnvsvk7jb9urqh3pymohtmlk7na1xxCe,oeDpwb8c57xlon7pwohatsk9fme5zvtn,1477965352,104.040088,30.660945,1.158169e+07,3.588794e+06
